#### Não precisa olhar esse código do Nixxla. Só fiz ele pra mostrar pra vocês os códigos do PredictionManager rodando

In [1]:
from mlforecast import MLForecast
import pandas as pd

In [2]:
path = r'C:\Users\micha\Documents\GitHub\Projeto-PET-Sauter\data\M5_full.parquet'

dados = pd.read_parquet(path)

In [3]:
dados['date'] = pd.to_datetime(dados['date'])

In [4]:
dados['year_month'] = dados['date'].dt.to_period('M')

In [5]:
dados['year_month'] = dados['year_month'].dt.to_timestamp()

In [6]:
df_general = dados.groupby('year_month')['value'].sum().reset_index().set_index('year_month')

In [7]:
df_hobbies = dados.groupby(['cat_id', 'year_month'])['value'].sum().reset_index().set_index('year_month')
df_hobbies = df_hobbies[df_hobbies['cat_id'] == 'HOBBIES']

C:\Users\micha\AppData\Local\Temp\ipykernel_9756\13977740.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_hobbies = dados.groupby(['cat_id', 'year_month'])['value'].sum().reset_index().set_index('year_month')


In [8]:
df_nixtla_general = pd.DataFrame({'ds': df_general.index, 'y': df_general['value'].values, 'unique_id' : 'general'})
df_nixtla_hobbies = pd.DataFrame({'ds': df_hobbies.index, 'y': df_hobbies['value'].values, 'unique_id': 'hobbies'})

df_nixtla = pd.concat([df_nixtla_general, df_nixtla_hobbies], ignore_index=True)

In [9]:
test_size = (int)(df_nixtla.shape[0]*0.2)

df_test = df_nixtla.tail(test_size)
df_train = df_nixtla.drop(df_test.index)

In [10]:
from sklearn.base import BaseEstimator

class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [12]:
selected_models = [
    DecisionTreeRegressor(),
    LinearRegression(),
    Naive()
]

In [13]:
ml_forecast = MLForecast(
    models = selected_models,
    freq = 'M',
    lags=[1,2,3,4,5,6]
)

In [14]:
ml_forecast.fit(df_train)

MLForecast(models=[DecisionTreeRegressor, LinearRegression, Naive], freq=M, lag_features=['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6'], date_features=[], num_threads=1)

##### Aqui é o que precisa ser salvo

In [15]:
predict_full = ml_forecast.predict(test_size)

C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\P

#### Vocês precisam usar isso aqui pra fazer o dashboard

##### Essa classe serve pra gerenciar as predições no nosso Banco de Dados de predição. Basicamente, serve como uma API pra salvar e consultar as predições feitas pelo Nixxla.

In [16]:
from PredictionsManager import PredictionsManager 

##### Esse path pro db eu criei pra testar a classe, então criem onde ficar melhor pra rodar o dashboard

In [17]:
db_path = 'predictions.db'

In [18]:
pred_manager = PredictionsManager(db_path=db_path)

Banco de dados conectado: predictions.db


##### Vejam que o meu DataFrame 'pred_full' tem as previsões de todos os modelos que eu selecionei pra cada uma dos unique_id

In [19]:
predict_full.head(5)

,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
0,general,2016-05-31,1083115.0,1.074594e+06,994982.0
1,general,2016-06-30,1051078.0,1.039590e+06,994982.0
2,general,2016-07-31,1013452.0,1.039560e+06,994982.0
3,general,2016-08-31,1117570.0,1.180017e+06,994982.0
4,general,2016-09-30,1031877.0,1.100387e+06,994982.0


In [20]:
predict_full.tail(5)

,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
47,hobbies,2015-12-31,94650.0,172619.078125,100565.0
48,hobbies,2016-01-31,94650.0,175924.296875,100565.0
49,hobbies,2016-02-29,100565.0,179369.406250,100565.0
50,hobbies,2016-03-31,104877.0,182163.156250,100565.0
51,hobbies,2016-04-30,104877.0,185714.093750,100565.0


##### Pra salvar essas previsões na base de dados basta passar esse DataFrame que o manager vai separar cada previsão em uma tabela com o nome correspondente ao 'unique_id' dela em 'predict_full'

In [21]:
pred_manager.save_predictions(predict_full)

Todas as previsões foram salvas em tabelas chamadas: ['general' 'hobbies']


##### Pra pegar as previsões, é só usar o método abaixo e pasasr o 'unique_id' correspondente a previsão que você quer e ele vai retornar um DataFrame que nem o abaixo.

In [22]:
prev_hobbies = pred_manager.get_prediction('hobbies')
prev_hobbies.head()

Previsões recuperadas com sucesso para: hobbies


,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
0,hobbies,2014-03-31,104877.0,102544.875000,100565.0
1,hobbies,2014-04-30,104877.0,108147.375000,100565.0
2,hobbies,2014-05-31,106435.0,116856.882812,100565.0
3,hobbies,2014-06-30,94650.0,115380.429688,100565.0
4,hobbies,2014-07-31,94650.0,120426.648438,100565.0


##### Depois de usar essa classe pra salvar/pegar algum dado no BD, lembrem de fechar a conexão.

In [23]:
pred_manager.close_connection()

Conexão fechada
